In [661]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder
from tabulate import tabulate

### Load Clusters


In [662]:
DATASET = "ne_id_ne_address"  # ne_id_ne_address
AGGREGATION_FIELD = "cluster_id2"  # cluster_id2, cluster_id3
CONCATENATION = "slogan"  # "slogan_network", "slogan_alarm_group"
MIN_SUPPORT = 0.005

ASSOCIATION_RULES_METRIC = "confidence"  # confidence, lift
MIN_THRESHOLD = 0.01
NUM_ASSOCIATION_RULES = 30

In [663]:
clusters_ne_id_loc_name = pd.read_parquet(
    "20240601_20240828_clusters_rdg_all_ne_id_loc_name_aggregation_column_first_occurrence_5min_5min_5min_delta.parquet"
)

In [664]:
clusters_ne_id_ne_address_first_three_octets = pd.read_parquet(
    "20240601_20240828_clusters_rdg_all_ne_id_ne_address_first_three_octets_aggregation_column_first_occurrence_5min_5min_5min_delta.parquet"
)

### Filter clusters with at least 2 devices inside


In [665]:
if AGGREGATION_FIELD == "cluster_id" or AGGREGATION_FIELD == "cluster_id3":
    pass
else:
    valid_clusters = clusters_ne_id_loc_name.groupby("cluster_id2").ne_id.nunique() >= 2
    clusters_ne_id_loc_name = clusters_ne_id_loc_name[
        clusters_ne_id_loc_name.cluster_id2.isin(
            list(valid_clusters[valid_clusters].index)
        )
    ]
    valid_clusters = (
        clusters_ne_id_ne_address_first_three_octets.groupby(
            "cluster_id2"
        ).ne_id.nunique()
        >= 2
    )
    clusters_ne_id_ne_address_first_three_octets = (
        clusters_ne_id_ne_address_first_three_octets[
            clusters_ne_id_ne_address_first_three_octets.cluster_id2.isin(
                list(valid_clusters[valid_clusters].index)
            )
        ]
    )

### Filter only important columns


In [666]:
clusters_ne_id_loc_name_filtered = clusters_ne_id_loc_name[
    [
        "cluster_id",
        "cluster_id2",
        "cluster_id3",
        "ne_type",
        "probable_cause",
        "alarm_group",
        "network",
        "first_occurrence",
        "alarm_id",
    ]
]

clusters_ne_id_ne_address_first_three_octets_filtered = (
    clusters_ne_id_ne_address_first_three_octets[
        [
            "cluster_id",
            "cluster_id2",
            "cluster_id3",
            "ne_type",
            "probable_cause",
            "alarm_group",
            "network",
            "first_occurrence",
            "alarm_id",
        ]
    ]
)

## Statistics


In [667]:
total_rows = len(clusters_ne_id_loc_name_filtered)

# Statistiche per il campo ne_type
filtered_rows_ne_type_nd = clusters_ne_id_loc_name_filtered[
    clusters_ne_id_loc_name_filtered["ne_type"] == "n/d"
]
num_filtered_rows_ne_type_nd = len(filtered_rows_ne_type_nd)
percentage_ne_type_nd = (num_filtered_rows_ne_type_nd / total_rows) * 100

filtered_rows_ne_type_na = clusters_ne_id_loc_name_filtered[
    clusters_ne_id_loc_name_filtered["ne_type"].isna()
]
num_filtered_rows_ne_type_na = len(filtered_rows_ne_type_na)
percentage_ne_type_na = (num_filtered_rows_ne_type_na / total_rows) * 100

# Statistiche per il campo probable_cause
filtered_rows_probable_cause_na = clusters_ne_id_loc_name_filtered[
    clusters_ne_id_loc_name_filtered["probable_cause"].isna()
]
num_filtered_rows_probable_cause_na = len(filtered_rows_probable_cause_na)
percentage_probable_cause_na = (num_filtered_rows_probable_cause_na / total_rows) * 100

# Statistiche per il campo alarm_group
filtered_rows_alarm_group_na = clusters_ne_id_loc_name_filtered[
    clusters_ne_id_loc_name_filtered["alarm_group"].isna()
]
num_filtered_rows_alarm_group_na = len(filtered_rows_alarm_group_na)
percentage_alarm_group_na = (num_filtered_rows_alarm_group_na / total_rows) * 100

# Statistiche per il campo network

filtered_rows_network_na = clusters_ne_id_loc_name_filtered[
    clusters_ne_id_loc_name_filtered["network"].isna()
]
num_filtered_rows_network_na = len(filtered_rows_network_na)
percentage_network_na = (num_filtered_rows_network_na / total_rows) * 100

# Statistiche per il campo network con valore 'rdg_others'
filtered_rows_network_rdg_others = clusters_ne_id_loc_name_filtered[
    clusters_ne_id_loc_name_filtered["network"] == "rdg_others"
]
num_filtered_rows_network_rdg_others = len(filtered_rows_network_rdg_others)
percentage_network_rdg_others = (
    num_filtered_rows_network_rdg_others / total_rows
) * 100

# Stampa delle statistiche
print(f"Percentuale di righe con ne_type uguale a 'n/d': {percentage_ne_type_nd:.2f}%")
print(f"Percentuale di righe con ne_type uguale a NA: {percentage_ne_type_na:.2f}%")
print(
    f"Percentuale di righe con probable_cause uguale a NA: {percentage_probable_cause_na:.2f}%"
)
print(
    f"Percentuale di righe con alarm_group uguale a NA: {percentage_alarm_group_na:.2f}%"
)
print(f"Percentuale di righe con network uguale a NA: {percentage_network_na:.2f}%")
print(
    f"Percentuale di righe con network uguale a 'rdg_others': {percentage_network_rdg_others:.2f}%"
)

Percentuale di righe con ne_type uguale a 'n/d': 94.91%
Percentuale di righe con ne_type uguale a NA: 0.00%
Percentuale di righe con probable_cause uguale a NA: 0.00%
Percentuale di righe con alarm_group uguale a NA: 0.00%
Percentuale di righe con network uguale a NA: 0.00%
Percentuale di righe con network uguale a 'rdg_others': 83.84%


In [668]:
total_rows = len(clusters_ne_id_ne_address_first_three_octets_filtered)

# Statistiche per il campo ne_type
filtered_rows_ne_type_nd = clusters_ne_id_ne_address_first_three_octets_filtered[
    clusters_ne_id_ne_address_first_three_octets_filtered["ne_type"] == "n/d"
]
num_filtered_rows_ne_type_nd = len(filtered_rows_ne_type_nd)
percentage_ne_type_nd = (num_filtered_rows_ne_type_nd / total_rows) * 100

filtered_rows_ne_type_na = clusters_ne_id_ne_address_first_three_octets_filtered[
    clusters_ne_id_ne_address_first_three_octets_filtered["ne_type"].isna()
]
num_filtered_rows_ne_type_na = len(filtered_rows_ne_type_na)
percentage_ne_type_na = (num_filtered_rows_ne_type_na / total_rows) * 100

# Statistiche per il campo probable_cause
filtered_rows_probable_cause_na = clusters_ne_id_ne_address_first_three_octets_filtered[
    clusters_ne_id_ne_address_first_three_octets_filtered["probable_cause"].isna()
]
num_filtered_rows_probable_cause_na = len(filtered_rows_probable_cause_na)
percentage_probable_cause_na = (num_filtered_rows_probable_cause_na / total_rows) * 100


filtered_rows_alarm_group_na = clusters_ne_id_ne_address_first_three_octets_filtered[
    clusters_ne_id_ne_address_first_three_octets_filtered["alarm_group"].isna()
]
num_filtered_rows_alarm_group_na = len(filtered_rows_alarm_group_na)
percentage_alarm_group_na = (num_filtered_rows_alarm_group_na / total_rows) * 100


filtered_rows_network_na = clusters_ne_id_ne_address_first_three_octets_filtered[
    clusters_ne_id_ne_address_first_three_octets_filtered["network"].isna()
]
num_filtered_rows_network_na = len(filtered_rows_network_na)
percentage_network_na = (num_filtered_rows_network_na / total_rows) * 100

# Statistiche per il campo network con valore 'rdg_others'
filtered_rows_network_rdg_others = (
    clusters_ne_id_ne_address_first_three_octets_filtered[
        clusters_ne_id_ne_address_first_three_octets_filtered["network"] == "rdg_others"
    ]
)
num_filtered_rows_network_rdg_others = len(filtered_rows_network_rdg_others)
percentage_network_rdg_others = (
    num_filtered_rows_network_rdg_others / total_rows
) * 100

# Stampa delle statistiche
print(f"Percentuale di righe con ne_type uguale a 'n/d': {percentage_ne_type_nd:.2f}%")
print(f"Percentuale di righe con ne_type uguale a NA: {percentage_ne_type_na:.2f}%")
print(
    f"Percentuale di righe con probable_cause uguale a NA: {percentage_probable_cause_na:.2f}%"
)
print(
    f"Percentuale di righe con alarm_group uguale a NA: {percentage_alarm_group_na:.2f}%"
)
print(f"Percentuale di righe con network uguale a NA: {percentage_network_na:.2f}%")
print(
    f"Percentuale di righe con network uguale a 'rdg_others': {percentage_network_rdg_others:.2f}%"
)

Percentuale di righe con ne_type uguale a 'n/d': 77.06%
Percentuale di righe con ne_type uguale a NA: 0.14%
Percentuale di righe con probable_cause uguale a NA: 0.00%
Percentuale di righe con alarm_group uguale a NA: 0.00%
Percentuale di righe con network uguale a NA: 8.64%
Percentuale di righe con network uguale a 'rdg_others': 69.86%


### Create new column based on the Aggregation


In [669]:
clusters_ne_id_loc_name_filtered = clusters_ne_id_loc_name_filtered.rename(
    columns={"probable_cause": "slogan"}
)

clusters_ne_id_ne_address_first_three_octets_filtered = (
    clusters_ne_id_ne_address_first_three_octets_filtered.rename(
        columns={"probable_cause": "slogan"}
    )
)


if CONCATENATION == "slogan":
    pass
elif CONCATENATION == "slogan_network":
    clusters_ne_id_loc_name_filtered[CONCATENATION] = (
        clusters_ne_id_loc_name_filtered["slogan"]
        + "_"
        + clusters_ne_id_loc_name_filtered["network"]
    )
    clusters_ne_id_ne_address_first_three_octets_filtered["slogan_network"] = (
        clusters_ne_id_ne_address_first_three_octets_filtered["slogan"]
        + "_"
        + clusters_ne_id_ne_address_first_three_octets_filtered["network"]
    )
elif CONCATENATION == "slogan_alarm_group":
    clusters_ne_id_loc_name_filtered[CONCATENATION] = (
        clusters_ne_id_loc_name_filtered["slogan"]
        + "_"
        + clusters_ne_id_loc_name_filtered["alarm_group"]
    )
    clusters_ne_id_ne_address_first_three_octets_filtered["slogan_alarm_group"] = (
        clusters_ne_id_ne_address_first_three_octets_filtered["slogan"]
        + "_"
        + clusters_ne_id_ne_address_first_three_octets_filtered["alarm_group"]
    )
else:
    raise ValueError(
        "Valore di CONCATENATION non valido. Deve essere 'slogan', 'slogan_network' o 'slogan_alarm_group'."
    )

### Drop columns with network == NA


In [670]:
if CONCATENATION == "slogan_network":
    clusters_ne_id_ne_address_first_three_octets_filtered = (
        clusters_ne_id_ne_address_first_three_octets_filtered.dropna(subset=["network"])
    )

## Preprocessing and FP-Growth


In [671]:
def create_baskets(data, aggregation_field):
    # baskets = data.groupby(aggregation_field)["slogan_netype"].apply(list)
    baskets = data.groupby(aggregation_field)[CONCATENATION].apply(list)
    te = TransactionEncoder()
    te_ary = te.fit(baskets).transform(baskets)
    basket_df = pd.DataFrame(te_ary, columns=te.columns_)

    return basket_df

In [672]:
def print_frequent_itemsets(frequent_itemsets):
    frequent_itemsets["itemsets"] = frequent_itemsets["itemsets"].apply(
        lambda x: ", ".join(list(x))
    )
    table = frequent_itemsets.values.tolist()
    return tabulate(
        table, headers=frequent_itemsets.columns, tablefmt="grid", showindex=True
    )

In [673]:
def print_first_N_rules(rules, N):
    rules_df = rules[
        ["antecedents", "consequents", "support", "confidence", "lift"]
    ].head(N)

    rules_df["antecedents"] = rules_df["antecedents"].apply(
        lambda x: ", ".join(list(x))
    )
    rules_df["consequents"] = rules_df["consequents"].apply(
        lambda x: ", ".join(list(x))
    )

    print("\nAssociation Rules:")
    print(tabulate(rules_df, headers="keys", tablefmt="pretty", showindex=True))

### ITEMSETS AND ASSOCIATION RULES


In [674]:
if DATASET == "ne_id_loc_name":
    data = clusters_ne_id_loc_name_filtered
elif DATASET == "ne_id_ne_address":
    data = clusters_ne_id_ne_address_first_three_octets_filtered
else:
    raise ValueError(
        "Valore di DATASET non valido. Deve essere 'ne_id_loc_name' o 'ne_id_ne_address'."
    )


# Create baskets from dataframe of clusters
basket_df = create_baskets(data, AGGREGATION_FIELD)
# Find frequent itemsets with fpgrowth
frequent_itemsets = fpgrowth(basket_df, min_support=MIN_SUPPORT, use_colnames=True)
# Sort itemsets by support
frequent_itemsets = frequent_itemsets.sort_values(by="support", ascending=False)


# Filter out itemsets with only one item
frequent_itemsets_copy = frequent_itemsets.copy()
frequent_itemsets_copy = frequent_itemsets_copy[
    frequent_itemsets_copy["itemsets"].apply(lambda x: len(x) > 1)
]

support_distribution = frequent_itemsets["support"].describe()
print("\nDistribuzione dei supporti:")
print(support_distribution)
print("\n")


print("AGGREGATION: " + AGGREGATION_FIELD + "\n")
print(print_frequent_itemsets(frequent_itemsets_copy))


# file_name = f"frequent_itemsets_{AGGREGATION_FIELD}_{CONCATENATION}_{DATASET}.xlsx"

# frequent_itemsets_copy.to_excel(file_name, index=False)


Distribuzione dei supporti:
count    166.000000
mean       0.028734
std        0.067741
min        0.005056
25%        0.007020
50%        0.010651
75%        0.020334
max        0.481549
Name: support, dtype: float64


AGGREGATION: cluster_id2

+-----+------------+----------------------------------------------------------------------------------------------------------------------------------+
|     |    support | itemsets                                                                                                                         |
+=====+============+==================================================================================================================================+
|   0 | 0.306939   | dns-node-down-alarms, dns-nodeunmanagable-alarms                                                                                 |
+-----+------------+-----------------------------------------------------------------------------------------------------------------------------

### Print Association Rules


In [675]:
rules = association_rules(
    frequent_itemsets, metric=ASSOCIATION_RULES_METRIC, min_threshold=MIN_THRESHOLD
)

print("AGGREGATION: " + AGGREGATION_FIELD + "\n")
print_first_N_rules(rules, NUM_ASSOCIATION_RULES)


rules["antecedents"] = rules["antecedents"].apply(lambda x: ", ".join(list(x)))
rules["consequents"] = rules["consequents"].apply(lambda x: ", ".join(list(x)))


# file_name = f"association_rules-{AGGREGATION_FIELD}-{CONCATENATION}-{DATASET}.xlsx"

# rules.to_excel(file_name, index=False)

AGGREGATION: cluster_id2


Association Rules:
+----+--------------------------------------------------+--------------------------------------------------+----------------------+---------------------+--------------------+
|    |                   antecedents                    |                   consequents                    |       support        |     confidence      |        lift        |
+----+--------------------------------------------------+--------------------------------------------------+----------------------+---------------------+--------------------+
| 0  |               dns-node-down-alarms               |            dns-nodeunmanagable-alarms            | 0.30693921463152235  | 0.9265995453069179  | 1.9242052666728782 |
| 1  |            dns-nodeunmanagable-alarms            |               dns-node-down-alarms               | 0.30693921463152235  | 0.6373994638069705  | 1.924205266672878  |
| 2  |            dns-nodeunmanagable-alarms            |               dns-lin

### Association Rules Sorted by Metrics


In [676]:
METRIC = "confidence"  # support, confidence, lift


# Funzione per ordinare le regole in base a una metrica specifica (support, confidence o lift)
def sort_rules(rules, metric="confidence", ascending=False):
    if metric not in ["support", "confidence", "lift"]:
        raise ValueError("La metrica deve essere 'support', 'confidence' o 'lift'.")
    return rules.sort_values(by=metric, ascending=ascending)


sorted_rules = sort_rules(rules, metric=METRIC)

print("\nRegole ordinate per: " + METRIC)


sorted_rules_df = pd.DataFrame(sorted_rules)
print(tabulate(sorted_rules_df, headers="keys", tablefmt="pretty", showindex=True))


Regole ordinate per: confidence
+------+------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------+-----------------------+-----------------------+-----------------------+----------------------+---------------------+-------------------------+--------------------+-----------------------+
|      |                                                antecedents                                                 |                                                consequents                                                 |  antecedent support   |  consequent support   |        support        |      confidence      |        lift         |        leverage         |     conviction     |     zhangs_metric     |
+------+------------------------------------------------------------------------------------------------------------+--------------------

## Exploration Functionalities


### Filter Itemsets containing a given string

In [677]:
FIELD = "down"


def filter_itemsets_by_string(itemsets, search_string):

    return itemsets[itemsets["itemsets"].apply(lambda x: search_string in x)]


filtered_itemsets = filter_itemsets_by_string(frequent_itemsets_copy, FIELD)


print(f"Itemsets che contengono '{FIELD}':")


print(tabulate(filtered_itemsets, headers="keys", tablefmt="pretty"))

Itemsets che contengono 'down':
+-----+-----------------------+----------------------------------------------------------------------------------------------------------------------------------+
|     |        support        |                                                             itemsets                                                             |
+-----+-----------------------+----------------------------------------------------------------------------------------------------------------------------------+
| 24  |  0.30693921463152235  |                                         dns-node-down-alarms, dns-nodeunmanagable-alarms                                         |
| 27  |  0.22668101129639592  |                                         dns-nodeunmanagable-alarms, dns-link-down-alarms                                         |
| 25  |  0.19892415277030662  |                                            dns-node-down-alarms, dns-link-down-alarms                                    

### Filter Ass. Rules containing a given string

In [678]:
# Funzione per filtrare le regole in base ad un particolare campo presente negli antecedents o nei consequents
def search_in_rules(rules, field, search_in):
    if search_in == "antecedents":
        return rules[rules["antecedents"].apply(lambda x: field in x)]
    elif search_in == "consequents":
        return rules[rules["consequents"].apply(lambda x: field in x)]
    elif search_in == "both":
        return rules[
            (rules["antecedents"].apply(lambda x: field in x))
            | (rules["consequents"].apply(lambda x: field in x))
        ]
    else:
        raise ValueError(
            "search_in deve essere 'antecedents', 'consequents', o 'both'."
        )

In [679]:
FIELD = "island"
SEARCH_IN = "consequents"  # antecedents, consequents, both
METRIC = "confidence"  # support, confidence, lift

searched_rules = search_in_rules(rules, field=FIELD, search_in=SEARCH_IN)


# sorted_rules = sort_rules(searched_rules, metric=METRIC)
# print("\nRegole ordinate per: " + METRIC)
# print_first_N_rules(sorted_rules, 1000)

print("Regole con <" + FIELD + "> negli " + SEARCH_IN + ":")
print(tabulate(searched_rules, headers="keys", tablefmt="pretty", showindex=True))

Regole con <island> negli consequents:
+-----+--------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+----------------------+----------------------+----------------------+----------------------+--------------------+------------------------+--------------------+----------------------+
|     |                                  antecedents                                   |                                    consequents                                    |  antecedent support  |  consequent support  |       support        |      confidence      |        lift        |        leverage        |     conviction     |    zhangs_metric     |
+-----+--------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+----------------------+----------------------+----------------------+----------

### Given an Association Rule, filter itemsets that contain all the elements of the rule


In [680]:
def find_rule_by_index(index, rules):
    specific_rule = rules.iloc[index]
    antecedent = specific_rule["antecedents"]
    consequent = specific_rule["consequents"]

    antecedent_list = [item.strip() for item in antecedent.split(",")]
    consequent_list = [item.strip() for item in consequent.split(",")]

    combined_rule = antecedent_list + consequent_list

    return antecedent, consequent, combined_rule



def find_itemsets_with_rule(itemsets, rule):

    rule_set = set(rule)

    return [itemset for itemset in itemsets if rule_set.issubset(set(itemset))]



def format_itemsets(itemsets):

    itemsets_list = itemsets.values.tolist()

    separated_itemsets_list = [
        [index] + [itemset[0]] + [item.strip() for item in itemset[1].split(", ")]
        for index, itemset in enumerate(itemsets_list)
    ]
    return separated_itemsets_list

In [681]:
# RULE = [
#     "dns-snmplinkup-alarms",
#     "dns-node-down-alarms",
#     "dns-ciscofruinserted-alarms",
#     "dns-snmpcoldstart-alarms",
#     "dns-nodeunmanagable-alarms",
# ]

RULE_INDEX = 842

antecedent, consequent, RULE = find_rule_by_index(RULE_INDEX, rules)


print(f"Regola di associazione scelta: {antecedent} => {consequent}")
print("\n")


itemsets = frequent_itemsets_copy
formatted_itemsets = format_itemsets(itemsets)
itemsets_with_rule = find_itemsets_with_rule(formatted_itemsets, RULE)


# Nice print of itemsets with rule
itemsets_with_rule_transformed = [
    [row[0], row[1], ", ".join(row[2:])] for row in itemsets_with_rule
]
headers = ["support", "itemsets"]
print(tabulate(itemsets_with_rule_transformed, headers=headers, tablefmt="grid"))

Regola di associazione scelta: dns-ciscofruinserted-alarms, dns-link-down-alarms => dns-snmplinkup-alarms


+-----+------------+--------------------------------------------------------------------------+
|     |    support | itemsets                                                                 |
+=====+============+==========================================================================+
| 124 | 0.00559441 | dns-snmplinkup-alarms, dns-ciscofruinserted-alarms, dns-link-down-alarms |
+-----+------------+--------------------------------------------------------------------------+


### Given an itemset, filter the clusters that contain the alarms


In [682]:
# Trova un itemset per indice e formatta ogni elemento come una lista di elementi
def find_itemset_by_index(index, itemsets):
    specific_itemset = itemsets.iloc[index]
    itemset_list = specific_itemset["itemsets"].split(", ")
    return itemset_list

In [683]:
def get_concatenation_by_cluster(clusters):
    cluster_slogans = []
    for cluster_id, cluster_data in clusters:
        if CONCATENATION in cluster_data.columns:
            slogans = list(set(cluster_data[CONCATENATION].tolist()))
            cluster_slogans.append({AGGREGATION_FIELD: cluster_id, CONCATENATION: slogans})
        else:
            print(f"Cluster {cluster_id} non ha il campo {CONCATENATION}")
    return pd.DataFrame(cluster_slogans)

In [684]:
# Funzione per filtrare i cluster_id in base all'ITEMSET
def filter_cluster_ids_by_itemset(cluster_slogans_df, itemset):
    filtered_cluster_ids = cluster_slogans_df[
        cluster_slogans_df[CONCATENATION].apply(lambda x: set(itemset).issubset(set(x)))
    ][AGGREGATION_FIELD]
    return filtered_cluster_ids

In [685]:
ITEMSET_INDEX = 2

ITEMSET = find_itemset_by_index(ITEMSET_INDEX, frequent_itemsets_copy)

print(f"ITEMSET scelto: {ITEMSET}")


common_columns = [AGGREGATION_FIELD, CONCATENATION, "first_occurrence", "alarm_id"]

if CONCATENATION != "slogan":
    common_columns.insert(1, "slogan")

# Seleziona i dati in base al valore di DATASET
if DATASET == "ne_id_loc_name":
    data = clusters_ne_id_loc_name_filtered[common_columns]
elif DATASET == "ne_id_ne_address":
    data = clusters_ne_id_ne_address_first_three_octets_filtered[common_columns]
else:
    raise ValueError(
        "Valore di DATASET non valido. Deve essere 'ne_id_loc_name' o 'ne_id_ne_address'."
    )

# Raggruppa i dati per cluster
clusters = data.groupby(AGGREGATION_FIELD)


# Ottieni i cluster con i loro slogan
cluster_slogans_df = get_concatenation_by_cluster(clusters)


# Filtra i cluster che contengono completamente l'ITEMSET
filtered_cluster_ids = filter_cluster_ids_by_itemset(cluster_slogans_df, ITEMSET)


print("Filtered Cluster IDs (aggregation_field = {}):".format(AGGREGATION_FIELD))
print(filtered_cluster_ids.tolist())
print("Number of filtered clusters: " + str(len(filtered_cluster_ids)))
print("Total number of clusters: " + str(len(clusters)))
print(
    "Percentage of clusters filtered: "
    + str(round((len(filtered_cluster_ids) / len(clusters)) * 100, 2))
    + "%"
)

CLUSTER_ID_TO_VIEW = 229


if CLUSTER_ID_TO_VIEW in filtered_cluster_ids.values:
    sample_cluster = data[data[AGGREGATION_FIELD] == CLUSTER_ID_TO_VIEW]
    print(f"{AGGREGATION_FIELD}: {CLUSTER_ID_TO_VIEW}")
    print(tabulate(sample_cluster, headers="keys", tablefmt="grid"))
else:
    print(
        f"{AGGREGATION_FIELD} {CLUSTER_ID_TO_VIEW} non trovato nella lista dei cluster filtrati."
    )

ITEMSET scelto: ['dns-node-down-alarms', 'dns-link-down-alarms']
Filtered Cluster IDs (aggregation_field = cluster_id2):
[229, 332, 336, 392, 398, 399, 400, 414, 419, 421, 2249, 6243, 6395, 6396, 6398, 6399, 6400, 6411, 6412, 6413, 6414, 6415, 6416, 6418, 6459, 6460, 6461, 6466, 6468, 6471, 6473, 6474, 6475, 6519, 6532, 6534, 6536, 6541, 6595, 6602, 6603, 6604, 6605, 6606, 6607, 6608, 6611, 6613, 6618, 6622, 6624, 6627, 6628, 6630, 6632, 6635, 6638, 6642, 6756, 6757, 6758, 6759, 6770, 6772, 6773, 6776, 6778, 7044, 7139, 7159, 7161, 7163, 7165, 7166, 7167, 7169, 7172, 7174, 7175, 7190, 7194, 7199, 7201, 7203, 7205, 7207, 7208, 7211, 7213, 7214, 7220, 7342, 7759, 7787, 7809, 7831, 7835, 7838, 7845, 7920, 7921, 7922, 7923, 7924, 7925, 7926, 7928, 7931, 7932, 7933, 7934, 7935, 7936, 7937, 7938, 7939, 7941, 7942, 7943, 7944, 7945, 7946, 7947, 7948, 7949, 7950, 7951, 7952, 7953, 7954, 7955, 7956, 7957, 7958, 7959, 7960, 7961, 7962, 7963, 7964, 7965, 7966, 7967, 7968, 7969, 7970, 7973, 7974, 

### Given a list of alarms, filter all the clusters that contain totally or not contain that alarms


In [686]:
ITEMSET = [
    "dns-rrgnosecondary-alarms_RrgNoSecondary",
    "dns-rrgnoprimary-alarms_RrgNoPrimary",
]
MODE = "in"  # in,  not_in


print(f"ITEMSET scelto: {ITEMSET}")
print(f"MODE: {MODE}")



common_columns = [AGGREGATION_FIELD, CONCATENATION, "first_occurrence", "alarm_id"]

if CONCATENATION != "slogan":
    common_columns.insert(1, "slogan")

# Seleziona i dati in base al valore di DATASET
if DATASET == "ne_id_loc_name":
    data = clusters_ne_id_loc_name_filtered[common_columns]
elif DATASET == "ne_id_ne_address":
    data = clusters_ne_id_ne_address_first_three_octets_filtered[common_columns]
else:
    raise ValueError(
        "Valore di DATASET non valido. Deve essere 'ne_id_loc_name' o 'ne_id_ne_address'."
    )


# Raggruppa i dati per cluster
clusters = data.groupby(AGGREGATION_FIELD)

# Ottieni i cluster con i loro slogan
cluster_concatenation_df = get_concatenation_by_cluster(clusters)



if MODE == "not_in":

    filtered_cluster_ids = cluster_concatenation_df[
        cluster_concatenation_df[CONCATENATION].apply(
            lambda x: not all(item in x for item in ITEMSET)
        )

    ][AGGREGATION_FIELD]
elif MODE == "in":
    filtered_cluster_ids = filter_cluster_ids_by_itemset(
        cluster_concatenation_df, ITEMSET
    )
else:
    raise ValueError("MODE deve essere 'in', 'not_in'.")



print("Filtered Cluster IDs:")
print(filtered_cluster_ids.tolist())
print("Number of filtered clusters: " + str(len(filtered_cluster_ids)))
print("Total number of clusters: " + str(len(clusters)))
print(
    "Percentage of clusters filtered: "
    + str(round((len(filtered_cluster_ids) / len(clusters)) * 100, 2))
    + "%"
)


CLUSTER_ID_TO_VIEW = 229


print("\n")



if CLUSTER_ID_TO_VIEW in filtered_cluster_ids.values:
    sample_cluster = data[data[AGGREGATION_FIELD] == CLUSTER_ID_TO_VIEW]
    print(f"{AGGREGATION_FIELD}: {CLUSTER_ID_TO_VIEW}")
    print(tabulate(sample_cluster, headers="keys", tablefmt="grid"))
else:
    print(
        f"{AGGREGATION_FIELD} {CLUSTER_ID_TO_VIEW} non trovato nella lista dei cluster filtrati."
    )

ITEMSET scelto: ['dns-rrgnosecondary-alarms_RrgNoSecondary', 'dns-rrgnoprimary-alarms_RrgNoPrimary']
MODE: in
Filtered Cluster IDs:
[]
Number of filtered clusters: 0
Total number of clusters: 9295
Percentage of clusters filtered: 0.0%


cluster_id2 229 non trovato nella lista dei cluster filtrati.


###


### Query with inclusion and exclusion conditions for alarms (e.g., all clusters that contain A and do NOT contain B)


In [687]:
def filter_clusters(clusters, inclusion_list, exclusion_list):
    inclusion_set = set(inclusion_list)
    exclusion_set = set(exclusion_list)

    filtered_cluster_ids = []
    for cluster_id, cluster_items in clusters:
        if inclusion_set.issubset(cluster_items) and exclusion_set.isdisjoint(
            cluster_items
        ):
            filtered_cluster_ids.append(cluster_id)
    return filtered_cluster_ids

In [688]:
inclusion_list = ["dns-nodeunmanagable-alarms_NodeUnmanagable", "dns-rrgnosecondary-alarms_RrgNoSecondary"]
exclusion_list = ["dns-snmpcoldstart-alarms_SNMPColdStart"]


common_columns = [AGGREGATION_FIELD, CONCATENATION, "first_occurrence", "alarm_id"]

if CONCATENATION != "slogan":
    common_columns.insert(1, "slogan")

# Seleziona i dati in base al valore di DATASET
if DATASET == "ne_id_loc_name":
    data = clusters_ne_id_loc_name_filtered[common_columns]
elif DATASET == "ne_id_ne_address":
    data = clusters_ne_id_ne_address_first_three_octets_filtered[common_columns]
else:
    raise ValueError(
        "Valore di DATASET non valido. Deve essere 'ne_id_loc_name' o 'ne_id_ne_address'."
    )

# Raggruppa i dati per cluster
grouped_data = (
    data.groupby(AGGREGATION_FIELD)
    .apply(lambda x: (x.name, set(x[CONCATENATION])))
    .tolist()
)


filtered_cluster_ids = filter_clusters(grouped_data, inclusion_list, exclusion_list)

print("Filtered Cluster IDs:")
print(filtered_cluster_ids)
print("Number of filtered clusters: " + str(len(filtered_cluster_ids)))
print("Total number of clusters: " + str(len(clusters)))
print(
    "Percentage of clusters filtered: "
    + str(round((len(filtered_cluster_ids) / len(clusters)) * 100, 2))
    + "%"
)

CLUSTER_ID_TO_VIEW = 516
print("\n")


if CLUSTER_ID_TO_VIEW in filtered_cluster_ids:
    sample_cluster = data[data[AGGREGATION_FIELD] == CLUSTER_ID_TO_VIEW]
    print(f"{AGGREGATION_FIELD}: {CLUSTER_ID_TO_VIEW}")
    print(tabulate(sample_cluster, headers="keys", tablefmt="grid", showindex=False))
else:
    print(
        f"{AGGREGATION_FIELD} {CLUSTER_ID_TO_VIEW} non trovato nella lista dei cluster filtrati."
    )

Filtered Cluster IDs:
[]
Number of filtered clusters: 0
Total number of clusters: 9295
Percentage of clusters filtered: 0.0%


cluster_id2 516 non trovato nella lista dei cluster filtrati.


C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_13356\513167518.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x.name, set(x[CONCATENATION])))


### Find Cluster by ID


In [689]:
common_columns = [AGGREGATION_FIELD, CONCATENATION, "first_occurrence", "alarm_id"]

if CONCATENATION != "slogan":
    common_columns.insert(1, "slogan")

# Seleziona i dati in base al valore di DATASET
if DATASET == "ne_id_loc_name":
    data = clusters_ne_id_loc_name_filtered[common_columns]
elif DATASET == "ne_id_ne_address":
    data = clusters_ne_id_ne_address_first_three_octets_filtered[common_columns]
else:
    raise ValueError(
        "Valore di DATASET non valido. Deve essere 'ne_id_loc_name' o 'ne_id_ne_address'."
    )


CLUSTER_ID_TO_VIEW = 333
print("\n")


sample_cluster = data[data[AGGREGATION_FIELD] == CLUSTER_ID_TO_VIEW]
print(f"{AGGREGATION_FIELD}: {CLUSTER_ID_TO_VIEW}")
print(tabulate(sample_cluster, headers="keys", tablefmt="grid"))




cluster_id2: 333
+-------+---------------+-------------------------------+---------------------+---------------------------------+
|       |   cluster_id2 | slogan                        | first_occurrence    | alarm_id                        |
+=======+===============+===============================+=====================+=================================+
| 26294 |           333 | dns-nodeunmanagable-alarms    | 2024-07-20 15:19:27 | EFMRDG;669bb960b8fdb700017cbc8c |
+-------+---------------+-------------------------------+---------------------+---------------------------------+
| 26271 |           333 | dns-node-down-alarms          | 2024-07-20 15:19:33 | EFMRDG;669bb967b8fdb700017cbc8d |
+-------+---------------+-------------------------------+---------------------+---------------------------------+
| 26281 |           333 | dns-rrgnoprimary-alarms       | 2024-07-20 15:20:45 | EFMRDG;669bb9afb8fdb700017cbc97 |
+-------+---------------+-------------------------------+------------